##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Sentiment Analysis

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/json_capabilities/Sentiment_Analysis.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

You will use the Gemini to extract sentiment scores of reviews.

In [ ]:
%pip install -U -q "google-genai>=1.0.0"

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Example

Start by defining how you want your JSON to be returned and which categories you would like to classify an item by. After that, go ahead and define some examples. In this case, you are trying to classify reviews as positive, neutral, or negative.

In [ ]:
import enum
from typing_extensions import TypedDict


class Magnitude(enum.Enum):
  WEAK = "weak"
  STRONG = "strong"


class Sentiment(TypedDict):
  positive_sentiment_score: Magnitude
  negative_sentiment_score: Magnitude
  neutral_sentiment_score: Magnitude


system_instruct = """
Generate each sentiment score probability (positive, negative, or neutral) for the whole text.
"""

In [ ]:
negative_review = "This establishment is an insult to the culinary arts, with inedible food that left me questioning the chef's sanity and the health inspector's judgment."
positive_review = "This restaurant is a true gem with impeccable service and a menu that tantalizes the taste buds. Every dish is a culinary masterpiece, crafted with fresh ingredients and bursting with flavor."
neutral_review = "The restaurant offers a decent dining experience with average food and service, making it a passable choice for a casual meal."

Take a look at each of the probabilities returned to see how each of these reviews would be classified by the Gemini model.

Helper function to generate content from sentiment llm:

In [ ]:
from google.genai import types

def generate_content(review):
    MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}
    return client.models.generate_content(
        model=MODEL_ID,
        contents=review,
        config=types.GenerateContentConfig(
            system_instruction=system_instruct,
            response_mime_type="application/json",
            response_schema=Sentiment,
        )
    )

In [ ]:
from pprint import pprint

response = generate_content(negative_review)
pprint(response.parsed)

{'negative_sentiment_score': <Magnitude.STRONG: 'strong'>,
 'neutral_sentiment_score': <Magnitude.WEAK: 'weak'>,
 'positive_sentiment_score': <Magnitude.WEAK: 'weak'>}


In [ ]:
response = generate_content(positive_review)
pprint(response.parsed)

{'negative_sentiment_score': <Magnitude.WEAK: 'weak'>,
 'neutral_sentiment_score': <Magnitude.WEAK: 'weak'>,
 'positive_sentiment_score': <Magnitude.STRONG: 'strong'>}


In [ ]:
response = generate_content(neutral_review)
pprint(response.parsed)

{'negative_sentiment_score': <Magnitude.WEAK: 'weak'>,
 'neutral_sentiment_score': <Magnitude.STRONG: 'strong'>,
 'positive_sentiment_score': <Magnitude.WEAK: 'weak'>}


## Summary
You have now used the Gemini API to analyze the sentiment of restaurant reviews using structured data. Try out other types of texts, such as comments under a video or emails.

Please see the other notebooks in this directory to learn more about how you can use the Gemini API for other JSON related tasks.